<a href="https://colab.research.google.com/github/s-im-ran/Machine-Learning/blob/main/11_HyperparameterTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Doing Hyperparameter Tuning in SVM model

In [ ]:
from sklearn import svm,datasets
iris=datasets.load_iris()

In [ ]:
import pandas as pd
df=pd.DataFrame(iris.data,columns=iris.feature_names)
df['flower']=iris.target
df['flower']=df['flower'].apply(lambda x: iris.target_names[x])
df[23:27]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
23,5.1,3.3,1.7,0.5,setosa
24,4.8,3.4,1.9,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
26,5.0,3.4,1.6,0.4,setosa


Doing the normal train-test split method

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(iris.data,iris.target,test_size=0.3)

In [ ]:
model=svm.SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.8888888888888888

Using the cross-val method

In [ ]:
from sklearn.model_selection import cross_val_score


In [ ]:
cross_val_score(svm.SVC(kernel='linear',C=10,gamma='auto'),iris.data,iris.target,cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

Using the GridSearchCV for hyperparameter tuning

In [ ]:
import numpy as np
kernels=['linear','rbf']
C=[1,10,20]
avg_scores={}
for kval in kernels:
  for cval in C:
    cv_scores=cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),iris.data,iris.target,cv=5)
    avg_scores[kval+'_'+str(cval)]=np.average(cv_scores)

avg_scores

{'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666,
 'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668}

In [ ]:
from sklearn.model_selection import GridSearchCV

clf=GridSearchCV(svm.SVC(gamma='auto'),{
    'C':[1,10,20],'kernel':['rbf','linear']
},cv=5,return_train_score=False)

clf.fit(iris.data,iris.target)
clf.cv_results_


{'mean_fit_time': array([0.00130024, 0.00082297, 0.00072346, 0.00064278, 0.00063314,
        0.00060277]),
 'std_fit_time': array([1.48251395e-04, 2.68774627e-05, 7.68490209e-05, 1.16068962e-04,
        1.35379771e-05, 9.05535483e-05]),
 'mean_score_time': array([0.00127697, 0.00081081, 0.00048542, 0.00041213, 0.00042548,
        0.00038471]),
 'std_score_time': array([1.15836701e-03, 4.73658467e-04, 1.15019113e-04, 2.89006592e-05,
        3.23644242e-05, 1.16162464e-05]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'ker

In [ ]:
df=pd.DataFrame(clf.cv_results_)

In [ ]:
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001300,0.000148,0.001277,0.001158,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000823,0.000027,0.000811,0.000474,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.000723,0.000077,0.000485,0.000115,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.000643,0.000116,0.000412,0.000029,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.000633,0.000014,0.000425,0.000032,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.000603,0.000091,0.000385,0.000012,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [ ]:
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


Doing HyperParameter Tuning using RandomizedGridSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rs=RandomizedSearchCV(svm.SVC(gamma='auto'),{'C':[1,10,20],'kernel':['linear','rbf'],
                                             },cv=5,return_train_score=False,n_iter=2)

In [ ]:
rs.fit(iris.data,iris.target)
pd.DataFrame(rs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001673,0.000770,0.001005,0.000748,rbf,20,"{'kernel': 'rbf', 'C': 20}",0.966667,1.0,0.9,0.966667,1.0,0.966667,0.036515,1
1,0.000935,0.000088,0.000663,0.000071,linear,20,"{'kernel': 'linear', 'C': 20}",1.000000,1.0,0.9,0.933333,1.0,0.966667,0.042164,2


Choosing the best model for our datasets

In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


In [ ]:
model_params={'svm':{'model':svm.SVC(gamma='auto'),'params':{'C':[1,10,20],'kernel':['rbf','linear']}}
              ,
              'random_forest':{'model':RandomForestClassifier(),'params':{'n_estimators':[1,5,10]}}
              ,
              'logistic_regression':{'model':LogisticRegression(solver='liblinear',multi_class='auto'),'params':{'C':[1,5,10]
              }
            }
          }

In [ ]:
scores=[]
for model_name,mp in model_params.items():
  clf=GridSearchCV(mp['model'],mp['params'],cv=5,return_train_score=False)
  clf.fit(iris.data,iris.target)
  scores.append({
      'model':model_name,
      'best_score':clf.best_score_,
      'best_params':clf.best_params_

  })

In [ ]:
df=pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 5}
2,logistic_regression,0.966667,{'C': 5}
